In [9]:
import logging
logging.basicConfig(filename="logs.log", level=logging.INFO)
import os.path
import os
import MDAnalysis as mda
from MDAnalysis.analysis import align, pca, rms

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
u = mda.Universe('neut.tpr', 'neut.pdb')

In [3]:
epj_mol = mda.Universe('epj.pdb')

In [15]:
epj_mol_resid = epj_mol.select_atoms('resname EPJ').resids[0]

In [12]:
epj_resids = np.unique(u.select_atoms('resname EPJ').resids)

In [36]:
epj_mol.select_atoms('resname EPJ and not name H*').names

array(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N1', 'C7', 'C8', 'C9', 'C10',
       'CL', 'N2', 'C11', 'LP1'], dtype=object)

In [29]:
epj_mol.select_atoms('not element H and resname EPJ and (resid 1011 or resid 402)').names

array(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N1', 'C7', 'C8', 'C9', 'C10',
       'CL', 'N2', 'C11'], dtype=object)

In [28]:
u.select_atoms('not element H and resname EPJ and (resid 1011 or resid 402)').names

array(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N1', 'C7', 'C8', 'C9', 'C10',
       'CL', 'N2', 'C11', 'LP1'], dtype=object)

In [43]:
u_merge_ag = u.select_atoms('not resname EPJ')
u_merge = mda.Merge(u_merge_ag)
for epj_resid in epj_resids:
    selection = f'not name H* and resname EPJ and (resid {epj_resid} or resid {epj_mol_resid})'
    print(selection)
    print(epj_mol.select_atoms(selection).names)
    print(u.select_atoms(selection).names)
    aligner_ref = align.AlignTraj(
        epj_mol, u,
        select=selection,
        in_memory=True,
        match_atoms=False
    ).run()
    u_merge = mda.Merge(u_merge.atoms, epj_mol.atoms)

not name H* and resname EPJ and (resid 1011 or resid 402)
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
not name H* and resname EPJ and (resid 1012 or resid 402)
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
not name H* and resname EPJ and (resid 1013 or resid 402)
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
not name H* and resname EPJ and (resid 1014 or resid 402)
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 'N2' 'C11'
 'LP1']
not name H* and resname EPJ and (resid 1015 or resid 402)
['C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'N1' 'C7' 'C8' 'C9' 'C10' 'CL' 

In [48]:
u_merge.dimensions = u.dimensions

In [49]:
u_merge.atoms.write('new.pdb')

/nethome/yzhuang/mdanalysis/package/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/nethome/yzhuang/mdanalysis/package/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/nethome/yzhuang/mdanalysis/package/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/nethome/yzhuang/mdanalysis/package/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/nethome/yzhuang/mdanalysis/package/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'tempfactors' Using default value of '0.0'
  warnin

In [ ]:
# And remove five Na+ from the system to keep it neutral.

# Binding site restraints

In [ ]:
binding_site_1 = u.select_atoms('name CA and byres (protein and around 3 segid seg_10_EPJ1)')
binding_site_2 = u.select_atoms('name CA and byres (protein and around 3 segid seg_11_EPJ2)')
with mda.selections.gromacs.SelectionWriter('index_fep.ndx', mode='w') as ndx:
    ndx.write(u.atoms, name='SYSTEM')
    ndx.write(u.select_atoms('protein or resname EPJ'), name='PROT')
    ndx.write(u.select_atoms('resname POPC CHL1'), name='MEMB')
    ndx.write(u.select_atoms('resname TIP3 SOD CLA'), name='SOL_ION')
    ndx.write(u.select_atoms('resname EPJ'), name='EPJ')
    ndx.write(u.select_atoms('segid seg_10_EPJ1'), name='EPJ1')
    ndx.write(u.select_atoms('segid seg_11_EPJ2'), name='EPJ2')
    ndx.write(binding_site_1, name='EPJ1_BS')
    ndx.write(binding_site_2, name='EPJ2_BS')

In [22]:
binding_site_1.ix

array([1594, 2441, 2452, 2476, 3123, 3159, 3169, 3223, 7034, 7465, 7921,
       7935, 7945, 8058, 8078, 8102])

: 

# Pore restraints

In [ ]:
#Pore resids: chain A G and resid 247 250 254 257 261 264 268
#Pore resids: chain C E I and resid 239 242 246 249 253 256 260

In [4]:
u = mda.Universe('em.pdb')

In [19]:
ch_1_pore = u.select_atoms('segid A and resid 247 250 254 257 261 264 268 and name CA')
ch_2_pore = u.select_atoms('segid C and resid 239 242 246 249 253 256 260 and name CA')
ch_3_pore = u.select_atoms('segid E and resid 239 242 246 249 253 256 260 and name CA')
ch_4_pore = u.select_atoms('segid G and resid 247 250 254 257 261 264 268 and name CA')
ch_5_pore = u.select_atoms('segid I and resid 239 242 246 249 253 256 260 and name CA')

In [27]:
from MDAnalysis.core import AtomGroup
from MDAnalysis.analysis.distances import distance_array
prime_notations = ['-1', '2', '6', '9', '13', '16', '20']
distance_arrays = {}
ag_arrays = {}
for prime_res, at1, at2, at3, at4, at5 in zip(
    prime_notations, ch_1_pore, ch_2_pore, ch_3_pore, ch_4_pore, ch_5_pore):
    print(f'Pore resids: prime {prime_res}, resname {at1.resname} {at2.resname} and resid {at1.resid} {at2.resid} {at3.resid} {at4.resid} {at5.resid}')
    ag_prime = AtomGroup([at1, at2, at3, at4, at5])
    ag_arrays[prime_res] = ag_prime
    distance_arrays[prime_res] = distance_array(ag_prime.positions, ag_prime.positions, box=u.dimensions)

Pore resids: prime -1, resname GLU GLU and resid 247 239 239 247 239
Pore resids: prime 2, resname THR THR and resid 250 242 242 250 242
Pore resids: prime 6, resname SER SER and resid 254 246 246 254 246
Pore resids: prime 9, resname LEU LEU and resid 257 249 249 257 249
Pore resids: prime 13, resname VAL VAL and resid 261 253 253 261 253
Pore resids: prime 16, resname LEU LEU and resid 264 256 256 264 256
Pore resids: prime 20, resname GLU LYS and resid 268 260 260 268 260


In [49]:
def write_pore_restraints_file(fname, prime_res, dist_mat, ag_arr):
    ag_arr_shift = np.roll(ag_arr, -2, axis=0)
    res_groups = [[at1.index+1, at2.index+1] for at1, at2 in zip(ag_arr, ag_arr_shift)]
    dist_groups = [dist_mat[i, j] for i,j in zip([0,1,2,3,4], [2,3,4,0,1])]
    with open(fname, 'w') as f:
        f.write(f'; Pore restraints for prime {prime_res}\n')
        f.write('[ bonds ]\n')
        f.write(';   i     j funct         lo        up1        up2    kdr\n')
        for res_group, dist in zip(res_groups, dist_groups):
            print(res_group, dist)
            f.write(f'{res_group[0]} {res_group[1]} 10 {(dist / 10):6f} {(dist / 10 +0.2):6f} {(dist / 10 +1.2):6f} 5000\n')


for prime_res in prime_notations:
    write_pore_restraints_file(f'toppar/pore_res_{prime_res}.itp', prime_res, distance_arrays[prime_res], ag_arrays[prime_res])

[3941, 15877] 13.356501769270437
[10012, 21814] 13.54450411681784
[15877, 27885] 13.738369917122442
[21814, 3941] 13.299814231852686
[27885, 10012] 13.749588110084607
[3997, 15931] 12.12820429850772
[10066, 21870] 12.347345409520399
[15931, 27939] 12.147663823714312
[21870, 3997] 12.376018807897989
[27939, 10066] 12.095757236668303
[4060, 15994] 14.499436495728057
[10129, 21933] 14.611938592832983
[15994, 28002] 14.441751658398056
[21933, 4060] 14.566677974486486
[28002, 10129] 14.43800919417531
[4106, 16040] 16.66162248728235
[10175, 21979] 16.62185413451094
[16040, 28048] 16.449311223760777
[21979, 4106] 17.004811817921492
[28048, 10175] 16.48996393577215
[4169, 16102] 16.652202181387143
[10237, 22042] 17.030669271954533
[16102, 28110] 16.48872477844872
[22042, 4169] 17.111670036787196
[28110, 10237] 16.886293487361094
[4224, 16157] 21.157485412673392
[10292, 22097] 21.456670574635826
[16157, 28165] 20.95151612354666
[22097, 4224] 21.29135917309957
[28165, 10292] 21.24265881920523
[4

In [47]:
with open('topol.top', '+a') as f:
    f.write('\n')
    f.write('#ifdef REST_PORE_ON\n')
    f.write('[ intermolecular_interactions]\n')
    for prime_res in prime_notations:
        f.write(f'#include "toppar/pore_res_{prime_res}.itp"\n')
    f.write('#endif\n')

# Binding site contact restraints

In [10]:
binding_site_1_n_side = u.select_atoms('protein and byres (protein and around 3 (segid seg_10_EPJ1 and name N1))')
binding_site_1_cl_side = u.select_atoms('protein and byres (protein and around 3 (segid seg_10_EPJ1 and name CL))')

binding_site_2_n_side = u.select_atoms('protein and byres (protein and around 3 (segid seg_11_EPJ2 and name N1))')
binding_site_2_cl_side = u.select_atoms('protein and byres (protein and around 3 (segid seg_11_EPJ2 and name CL))')

with mda.selections.gromacs.SelectionWriter('index_contact.ndx', mode='w') as ndx:
    ndx.write(u.atoms, name='SYSTEM')
    ndx.write(u.select_atoms('protein or resname EPJ'), name='PROT')
    ndx.write(u.select_atoms('resname POPC CHL1'), name='MEMB')
    ndx.write(u.select_atoms('resname TIP3 SOD CLA'), name='SOL_ION')
    ndx.write(u.select_atoms('resname EPJ'), name='EPJ')
    ndx.write(u.select_atoms('segid seg_10_EPJ1'), name='EPJ1')
    ndx.write(u.select_atoms('segid seg_11_EPJ2'), name='EPJ2')
    ndx.write(u.select_atoms('segid seg_10_EPJ1 and name N1'), name='EPJ1_N1')
    ndx.write(u.select_atoms('segid seg_10_EPJ1 and name CL'), name='EPJ1_CL')
    ndx.write(u.select_atoms('segid seg_11_EPJ2 and name N1'), name='EPJ2_N1')
    ndx.write(u.select_atoms('segid seg_11_EPJ2 and name CL'), name='EPJ2_CL')
    ndx.write(binding_site_1_n_side, name='EPJ1_BS_n_side')
    ndx.write(binding_site_1_cl_side, name='EPJ1_BS_cl_side')
    ndx.write(binding_site_2_n_side, name='EPJ2_BS_n_side')
    ndx.write(binding_site_2_cl_side, name='EPJ2_BS_cl_side')    